In [3]:
import numpy as np
import os
import torch
from time import time
from torch.autograd import Variable
from PIL import Image

import sys
sys.path.append('../')
sys.path.append('/Users/Simona/tile2vec/src')
import tilenet
from tilenet import make_tilenet
import resnet 
from resnet import ResNet18
#from src.tilenet import make_tilenet
#from src.resnet import ResNet18
import torchvision
from torchvision import datasets, transforms

In [ ]:
%cd '/Users/Simona/'

In [4]:
import Classifier_v4
from Classifier_v4 import RF
from Classifier_v4 import LR
from Classifier_v4 import MLP

## Step 1. Loading pre-trained model
In this step, we will initialize a new TileNet model and then load the pre-trained weights.

In [ ]:
# Setting up model
in_channels = 4
z_dim = 512
cuda = torch.cuda.is_available()
# tilenet = make_tilenet(in_channels=in_channels, z_dim=z_dim)
# Use old model for now
tilenet = ResNet18()
if cuda: tilenet.cuda()

In [ ]:
%cd tile2vec/models

In [ ]:
# Load parameters
model_fn = '../models/naip_trained.ckpt'
print(model_fn)
checkpoint = torch.load(model_fn,map_location=torch.device('cpu'))
tilenet.load_state_dict(checkpoint)
tilenet.eval()

## Step 2. Embed NAIP tiles
In this step, we'll use TileNet to embed the NAIP tiles provided in `tile2vec/data/tiles`. There are 1000 tiles in total, named `1tile.npy` through `1000tile.npy`.

In [5]:
from torchvision import datasets, transforms

In [6]:
# Set the image transforms
train_transform = transforms.Compose([transforms.Resize(100),
                                    #transforms.RandomResizedCrop(scale=(0.16, 1), ratio=(0.75, 1.33), size=64),
                                    #transforms.RandomHorizontalFlip(0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

val_transform = transforms.Compose([transforms.Resize(100),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])




In [7]:
train_dataset =  datasets.ImageFolder('/Users/Simona/Fresno_Area/train', transform=train_transform)
val_dataset =  datasets.ImageFolder('/Users/Simona/Fresno_Area/val', transform=val_transform)
test_dataset =  datasets.ImageFolder('/Users/Simona/Fresno_Area/test', transform=val_transform)

# Train embeddings

In [ ]:
# Embed tiles
from torch.autograd import Variable
n_tiles = len(train_dataset)
z_dim = 512
X_train = np.zeros((n_tiles, z_dim))
for idx in range(n_tiles):
    tile = train_dataset[idx][0]
    tile = tile.numpy()
    tile = np.concatenate((tile, np.zeros((1, 100, 100))), axis=0)
    tile = np.expand_dims(tile, axis=0)
    # Scale to [0, 1]
    tile = tile / 255
    # Embed tile
    tile = torch.from_numpy(tile).float()
    tile = Variable(tile)
    z = tilenet.encode(tile)
    z = z.data.numpy()
    #print(z.shape)
    X_train[idx,:] = z

In [ ]:
np.savetxt('/Users/Simona/Fresno_Area/X_train_tile2vec',X_train)

In [8]:
X_train = np.loadtxt('/Users/Simona/Fresno_Area/X_train_tile2vec')
X_test = np.loadtxt('/Users/Simona/Fresno_Area/X_test_tile2vec')

# Test embeddings

In [ ]:
# Embed tiles
from torch.autograd import Variable
n_tiles = len(test_dataset)
z_dim = 512
X_test = np.zeros((n_tiles, z_dim))
for idx in range(n_tiles):
    tile = test_dataset[idx][0]
    tile = tile.numpy()
    tile = np.concatenate((tile, np.zeros((1, 100, 100))), axis=0)
    tile = np.expand_dims(tile, axis=0)
    # Scale to [0, 1]
    tile = tile / 255
    # Embed tile
    tile = torch.from_numpy(tile).float()
    tile = Variable(tile)
    z = tilenet.encode(tile)
    z = z.data.numpy()
    #print(z.shape)
    X_test[idx,:] = z

In [ ]:
np.savetxt('/Users/Simona/Fresno_Area/X_test_tile2vec',X_test)

# Classifier

In [28]:
import random
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from numpy.linalg import norm
import random
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression


def build_pairs_training(train_emb):

    train_pairs = train_emb.shape[0]
    same = []
    same_labels = []

    for i in range(0,train_pairs,2):
        #print(i,i+1)
        same.append((train_emb[i],train_emb[i+1]))
        same_labels.append(1)
    #print("same",len(same))

    #naip even num, sen odd num
    diff = []
    diff_labels = []

    for i in range(0,train_pairs,2):
        j = random.randrange(1,(train_pairs-2),2)
        if i != j and abs(i-j)>1:
            #print(i,j)
            diff.append((train_emb[i],train_emb[j]))
            diff_labels.append(0)
        else:
            j +=2
            #print("diff",i,j)
            diff.append((train_emb[i],train_emb[j]))
            diff_labels.append(0)

    train_emb= same+diff
    train_lab = same_labels+diff_labels

    train_emb  = np.asarray(train_emb)
    train_lab   = np.asarray(train_lab)
    indices = np.arange(train_emb.shape[0])
    np.random.shuffle(indices)

    train_emb  = train_emb[indices]
    train_lab  = train_lab[indices]
     
    #print(train_emb.shape)
    train_emb = train_emb.reshape(train_emb.shape[0],train_emb.shape[1]*train_emb.shape[2])
    #print(train_emb.shape)
    #train_emb = train_emb.reshape(-1, 1)
    #train_emb = train_emb.reshape(-1, 1)

    return train_emb , train_lab

def build_pairs_testing(test_emb):

    same = []
    same_labels = []

    test_pairs = test_emb.shape[0]

    for i in range(0,test_pairs,2):
        same.append((test_emb[i],test_emb[i+1]))
        same_labels.append(1)

    #naip even num, sen odd num
    diff = []
    diff_labels = []

    for i in range(0,test_pairs,2):
        j = random.randrange(1,(test_pairs-2),2)
        if i != j and abs(i-j)>1:
            #print(i,j)
            diff.append((test_emb[i],test_emb[j]))
            diff_labels.append(0)
        else:
            j +=2
            #print("diff",i,j)
            diff.append((test_emb[i],test_emb[j]))
            diff_labels.append(0)

    test_emb= same+diff
    test_lab = same_labels+diff_labels

    test_emb = np.asarray(test_emb)
    test_lab  = np.asarray(test_lab)
    indices = np.arange(test_emb.shape[0])
    np.random.shuffle(indices)

    test_emb = test_emb[indices]
    test_lab = test_lab[indices]
    
    test_emb = test_emb.reshape(test_emb.shape[0],test_emb.shape[1]*test_emb.shape[2])

    #test_emb = test_emb.reshape(-1, 1)
    #test_emb=  test_emb.reshape(-1, 1)

    return test_emb, test_lab

def RF(X_train,X_test,n_trials = 100):
    accs = np.zeros((n_trials,))

    for i in range(n_trials):
        # Splitting data and training RF classifer
        X_tr, y_tr = build_pairs_training(X_train)
        X_te, y_te =  build_pairs_testing(X_test)
        rf = RandomForestClassifier()
        rf.fit(X_tr, y_tr)
        accs[i] = rf.score(X_te, y_te)
    print('Mean accuracy: {:0.4f}'.format(accs.mean()))
    print('Standard deviation: {:0.4f}'.format(accs.std()))

def LR(X_train,X_test,n_trials = 100):
      accs = np.zeros((n_trials,))
      for i in range(n_trials):
          # Splitting data and training RF classifer
          X_tr, y_tr = build_pairs_training(X_train)
          X_te, y_te =  build_pairs_testing(X_test)
          reg = LogisticRegression(max_iter = 4000)
          #print(X_tr.shape)
          #print(y_tr.shape)
          #print(X_te.shape)
          #print(y_te.shape)
          reg.fit(X_tr, y_tr)
          accs[i] = reg.score(X_te, y_te)
      print('Mean accuracy: {:0.4f}'.format(accs.mean()))
      print('Standard deviation: {:0.4f}'.format(accs.std()))

def MLP(X_train,X_test,n_trials = 100):
      accs = np.zeros((n_trials,))

      for i in range(n_trials):
          # Splitting data and training RF classifer
          X_tr, y_tr = build_pairs_training(X_train)
          X_te, y_te =  build_pairs_testing(X_test)
          clf = MLPClassifier(max_iter=300)
          clf.fit(X_tr, y_tr)
          accs[i] = clf.score(X_te, y_te)
      print('Mean accuracy: {:0.4f}'.format(accs.mean()))
      print('Standard deviation: {:0.4f}'.format(accs.std()))


In [29]:
#RF(X_train,X_test)
LR(X_train,X_test)
#MLP(X_train,X_test)

Mean accuracy: 0.4969
Standard deviation: 0.0316


# Distance Plot

In [ ]:
from numpy.linalg import norm
distance_to_ = []
for i in range(0,X_test.shape[0]):
  sublist = []
  for j in range(0,X_test.shape[0]):
    labels = tuple([test_dataset[i][1],test_dataset[j][1]])
    sublist.append(tuple([norm(X_test[i]-X_test[j]),labels]))
  distance_to_.append(sublist)

In [ ]:
x_neg = []
x_pos = []
x_arr_neg = np.asarray([0]*len(test_dataset))
x_arr_pos = np.asarray([0]*len(test_dataset))
x_arr_neg_list = []
x_arr_pos_list = []
x_arr_labels_list = []
x_arr_labels_list_n = []

for i in range(0,int(len(distance_to_)/2)):
  x_n_ = []
  x_p_ = []
  x_p_labels = []
  x_n_labels = []
  x_n_labels_raw = []
  for j in range(0,int(len(distance_to_)/2)):
    if distance_to_[i*2+1][j*2][1][1] == i:
      x_p_.append(distance_to_[i*2+1][j*2][0]) 
      x_p_labels.append(i)
    else:
      x_n_.append(distance_to_[i*2+1][j*2][0])
      x_n_labels.append((i*2+1,j*2))
      x_n_labels_raw.append((i,j))
  x_arr_neg_ = np.asarray(x_n_)
  x_arr_pos_ = np.asarray(x_p_)
  x_arr_neg_list.append(x_arr_neg_)
  #x_arr_pos = x_arr_pos + x_arr_pos_
  x_arr_pos_list.append(x_arr_pos_)
  x_arr_labels_list.append(x_p_labels)
  x_arr_labels_list_n.append(x_n_labels_raw)
  x_mean_neg = np.mean(x_arr_neg, axis=0)
  x_mean_pos = np.mean(x_arr_pos, axis=0)
  x_std_neg = np.std(x_arr_neg, axis=0)
  x_std_pos = np.std(x_arr_pos, axis=0)

In [ ]:
import seaborn as sns, numpy as np
import matplotlib.pyplot as plt
sns.set()
x = x_arr_neg_list
x_2 =  x_arr_pos_list
ax = sns.distplot(x,label="different location")
ax = sns.distplot(x_2,label="same location")
ax.set(xlabel='Distance', ylabel='Distribution')
plt.title("Distance between tile2vec embeddings") 
plt.legend()
plt.show()